# Segmentation: on the cluster

### summary of segmentation workflow

##### first, transfer the raw image data from the hcbi server to the local hard drive or external hard drive
##### use rsync to transfer the raw image data from the local drive to /n/gaudet_lab/users/emay/microscopy/ on the rc cluster
##### create an output directory inside the experiment folder on the cluster
##### use segmentation.ipynb (using the custom jupyter lab setup in an interactive node) to load the raw data in
##### assign sample names to images appropriately
##### segment with cellpose; visually inspect the segmentation and adjust parameters as necessary
##### save the segmentation labels and sample names to the output directory in the data's folder on the cluster
##### use rsync to transfer the cluster folder containing the data, labels and sample names back to the local drive, and start this notebook from there

In [1]:
# set up directories and paths

n = '240201'
# n = '240131'
# n = '240113'
# n = '240112'
# n = '240111'
# n = '231220'
# n = '231215'
# n = '231117'
# n = '231014'
# n = '231013'
# n = '231011'
# n = '230929'
# n = '230927'
# n = '230920'
# n = '230916'

p = './'+n+'_LSM880/'
o = p+'out/'

In [2]:
# import packages

import numpy as np
import fraplib as fl
import pandas as pd
from matplotlib import pyplot as plt
# import seaborn as sns
import xarray as xr
import microutil as mu
# import matplotlib.patches as pch
from mpl_interactions import hyperslicer
from scipy import ndimage as ndi
# import random
import napari

%matplotlib ipympl

# load data

In [3]:
h, _ = fl.batchread(p)

# I took extra images in addition to ones for the assay for some experiments; select only the assay images for this analysis
if n == '231004': 
    h = {k : h[k] for k in ['Image 6.czi', 'Image 8.czi']}
elif n == '231006':
    h = {k : h[k] for k in ['Image 1.czi', 'Image 4.czi']}

In [4]:
exptid = [i[:-4] for i in list(h.keys())]
exptid.sort()

k = list(h.keys())
k.sort()
k

['Image 1.czi', 'Image 2.czi']

In [5]:
name = k[0]
name

'Image 1.czi'

In [6]:
chnames = fl.channel_label(h[name])
chnames

['mCherry', 'Transmitted Light', 'FITC']

In [7]:
images = np.concatenate([h[j].asarray().squeeze() for j in k], axis = 0)
images.shape

(48, 3, 2048, 2048)

In [8]:
samples = pd.read_csv(o+'samples_'+n+'.txt', header = None)[0].values
print(samples), len(samples)

['gC3FL+' 'gA11dC+' 'mock' 'gA11dC-' 'gC3dsdC' 'gC3ec56' 'gC3FL-'
 'gC3FL-unstained' 'a4dCmCh+' '14xdC+' '22xdC+' 'a4+14x' 'a4+22x'
 'a4+gA11dC-' '14x+gA11dC-' '22x+gA11dC-']


(None, 16)

In [9]:
images.shape

(48, 3, 2048, 2048)

In [10]:
reps = 3

ximages = xr.DataArray(
    images.reshape(int(images.shape[0]/reps),reps,*images.shape[-3:]), 
    dims = list('SRCYX'), 
    coords = { 'S' : samples, 'C' : fl.channel_label(h[list(h.keys())[0]]) } )

# load segmentation labels

### summary of labels editing workflow

##### first, load the labels in from the hard drive as a numpy array
##### note: labels were saved to the hard drive after automated segmentation on the cluster (using a GPU) via cellpose
##### reshape labels and make them as xarray to match ximages
##### make vxl (viewer xlabels) by reshaping the xlabels into a 3D array from a 4D array
##### make vxi (viewer ximages) by selecting just the transmitted light channel from ximages (.sel()) and reshaping it into a 3D array from a 4D array
##### note: vxl and vxi as 3D arrays are necessary to make the fill option work in the napari viwer
##### open a napari viewer, load vxi and vxl
##### edit the labels manually (I use Astropad and ipad to draw/edit the labels)
##### bring the manually-edited labels back from the viewer as a new "labels" variable
##### reshape labels the same may the images were originally reshaped to a 4D array to maintain the correct Sample and Replicate matching
##### save the new 4D labels array as .npy
##### create a new xlabels object with the new labels and dims, coords matching those of ximages
##### proceed on to image processing section

In [11]:
labels = np.load(o+'labels_'+n+'.npy')

labels_cmap = plt.cm.viridis.copy()
labels_cmap.set_under(alpha = 0)

if n == '231115':
    
    labels = np.concatenate( [
            *labels[:33,...].reshape(int(labels[:33,...].shape[0]/reps),reps,*labels.shape[-2:]),
            np.stack([*labels[-2:,...],np.empty(labels.shape[-2:]) * np.nan]) ], axis = 0 )
    
    ximages = xr.concat([
        ximages[:11],
        xr.concat([
            ximages[11,:-1,...],
            ximages[11,-1].where(np.zeros(ximages.shape[-3:], dtype = bool)) ], dim = 'R') ], dim = 'S' )

xlabels = xr.DataArray(
    labels.reshape(int(labels.shape[0]/reps),reps,*labels.shape[-2:]),
    dims = list('SRYX'),
    coords = { 'S' : ximages.S }
)

In [12]:
ximages.sel({'C' : 'Transmitted Light'}).shape

(16, 3, 2048, 2048)

In [13]:
vxl = xlabels.values.reshape((xlabels.shape[0]*xlabels.shape[1], xlabels.shape[-2], xlabels.shape[-1]))
vxi = ximages.sel({'C' : 'Transmitted Light'}).values.reshape((xlabels.shape[0]*xlabels.shape[1], xlabels.shape[-2], xlabels.shape[-1]))

In [14]:
viewer = napari.view_image(vxi, name = n+'_bfs')
viewer.add_labels(vxl)

<Labels layer 'vxl' at 0x16a5ae6a0>

#### if desired, modify labels in place in napari

In [15]:
# labels = viewer.layers[1].data
# labels = labels.reshape(int(images.shape[0]/reps),reps,*images.shape[-2:])
# labels.shape

In [16]:
# new_labels = labels.data
# np.save(o+'new_labels_'+n+'.npy', new_labels)
# new_labels.shape

In [17]:
# xlabels = xr.DataArray(new_labels, dims = xlabels.dims, coords = xlabels.coords)

# begin image processing stages

In [18]:
ximages_allchannels = ximages

In [19]:
ximages = ximages.drop_sel(C = 'Transmitted Light')

In [20]:
from scipy.stats import linregress
# import seaborn as sns

In [21]:
if n == '231115':
    print(samples[:12])
else:
    print(samples)

['gC3FL+' 'b17FL+' 'aC2FL+' 'gB6dC+' 'a4+gC3FL-' 'mock' 'a4dCmCh+'
 'gC3FL-' '5xdC+' '10xdC+' '12xdC+' 'gB6dsdC' 'gA11dsdC' 'aC2dsdC'
 'gA3dsdC' 'b17dsdC']


In [22]:
# bleedthrough correction (citation: Jost et al. 2019 J. Cell. Biol.)

bt = 0.005 # calculated from universal_bleedthrough_analysis_part2.ipynb

btc = ximages.sel({'C':'FITC'}) - bt*ximages.sel({'C':'mCherry'}) # correct FITC channel
btc = xr.concat([ximages.sel({'C':'mCherry'}), btc], 'C') # new xarray containing mCherry and corrected FITC channels
btc = btc.assign_coords({'C':['mCherry', 'FITC']}) # assign coordinate names
btc = btc.transpose(*ximages.dims) # rearrange dimensions to match the original order from ximages

btc = ximages

# mean vs median of background pixels
fig, axs = plt.subplots(2,1, layout = 'constrained', figsize = (6.5, 6))

bkgd = btc.where(xlabels==0)

for m, ch in enumerate(['mCherry', 'FITC']):
    arr = np.array(bkgd.sel({'C':ch})).ravel()
    arr = arr[~np.isnan(arr)]
    axs[m].hist(arr, bins = 1000)
    axs[m].semilogy()
    axs[m].set_title(ch)
    axs[m].axvline(arr.mean(), color = 'k')
    axs[m].axvline(np.median(arr), color = 'darkblue')
    axs[m].set_xlim(-100,300)
plt.show()

# median slightly less weighted by right-tail outliers

In [23]:
# background subtraction: for each image, subtract from all pixels the median value of non-cell pixels
# bkgd = ximages.where(xlabels==0)
bkgd = btc.where(xlabels==0)
mdbkgd = bkgd.median(list("YX"))
# mnbkgd = bkgd.mean(list("YX"))
# bgims = ximages - mdbkgd # median background subtraction
bgims = btc - mdbkgd # median background subtraction

In [24]:
# consider pixel normalization to mock transfected sample in mCherry and FITC channels (code in initial_processing.ipynb)
# excluding pixel normalization here

In [25]:
# computing cell-wise properties
avgs = mu.single_cell.average(xlabels.to_dataset(name='labels'),bgims).to_series()
areas = mu.single_cell.area(xlabels.to_dataset(name='labels')).to_series()

In [26]:
# excluding any cells with saturated pixels from analysis
sat = xlabels.where(ximages.loc[:,:,'mCherry'] == 2**16-1).dropna(dim='S',how = 'all')
indS = []
indR = []
cellID = []
for s in sat:
    for x,r in enumerate(s):
        sat_cells = np.unique(r)
        for num in sat_cells:
            indS.append(str(s.S.data))
            indR.append(x)
            if np.isnan(num):
                cellID.append(num)
            else:
                cellID.append(int(num))

sat = xlabels.where(ximages.loc[:,:,'FITC'] == 2**16-1).dropna(dim='S',how = 'all')
for s in sat:
    for x,r in enumerate(s):
        sat_cells = np.unique(r)
        for num in sat_cells:
            indS.append(str(s.S.data))
            indR.append(x)
            if np.isnan(num):
                cellID.append(num)
            else:
                cellID.append(int(num))

if indS or indR or cellID:
    drop_sat = pd.MultiIndex.from_tuples(zip(indS, indR, cellID)).dropna()
else:
    drop_sat = []

avgs = avgs.unstack('C').drop(drop_sat).stack().dropna()
areas = areas.drop(drop_sat).astype(int).dropna()

# drop_sat

# cellwise-processed data

In [27]:
data = pd.concat([areas, avgs.unstack('C')], axis = 1)
data.columns = ['area',*avgs.unstack('C').columns]
multidata = data.reset_index()
multidata['E'] = n

multidata = multidata.set_index(list('ESR')+['CellID']).dropna(how = 'any')
multidata.to_csv('./cellwise_values/without_bleedthrough_correction/'+n+'.csv', index_label=multidata.index.names)

# data = pd.read_csv('./cellwise_values/without_bleedthrough_correction/'+n+'.csv', index_col = list('ESR')+['CellID'])
# data